In [32]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [33]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [34]:
from typing import List

from models.llm.chain import BaseChain

In [35]:
template = '''
I want you to look at the draft and extract a part that corresponds to the user's query.
The extracted part should not differ from the original draft.
The extracted part is organized in paragraph.
Just say the extracted part. But if you don't have any, don't say anything and don't explain.

draft:
{draft}

user query: {query}

Extracted part:
'''

In [44]:
class DraftChunkChain(BaseChain):
    def __init__(self, 
                draft_chunk_template=None, 
                input_variables:List[str]=None,
                draft_chunk_template_path='../openai_skt/models/templates/draft_chunk_prompt.json', 
                model='gpt-3.5-turbo-16k', 
                verbose=False) -> None:
        super().__init__(template=draft_chunk_template, input_variables=input_variables, template_path=draft_chunk_template_path, model=model, verbose=verbose)

    def run(self, draft:str=None, query:str=None):
        return super().run(draft=draft, query=query)
    
    async def arun(self, draft:str=None, query:str=None):
        return await super().arun(draft=draft, query=query)

In [45]:
draft_chunk_chain = DraftChunkChain(draft_chunk_template=template, input_variables=['draft', 'query'], verbose=True)

### Tool 설정

In [46]:
from typing import Optional, Type, Any
from pydantic import BaseModel

from langchain.tools import BaseTool

In [47]:
class DraftChunkTool(BaseTool):
    name = "draft_chunk_tool"
    description = "A tool to extract a part of draft that corresponds to the user's query."
    args_schema: Optional[Type[BaseModel]] = None
    """Pydantic model class to validate and parse the tool's input arguments."""

    draft_chunk_chain: Any 

    def __init__(self, draft_chunk_chain) -> None:
        super().__init__()
        self.draft_chunk_chain = draft_chunk_chain

    def _run(self, draft:str, query:str) -> dict:
        result = self.draft_chunk_chain.run(draft=draft, query=query)
        return result
    
    async def _arun(self, draft:str, query:str) -> dict:
        result = await self.draft_chunk_chain.arun(draft=draft, query=query)
        return result

In [48]:
draft_chunk_tool = DraftChunkTool(draft_chunk_chain=draft_chunk_chain)

In [49]:
draft = '''
# 서론

이순신 장군에 대한 간략한 소개

이순신 장군은 조선시대의 영웅이자 조선해군의 명장으로, 그의 삶과 전투 업적은 오늘날 대한민국의 국가 정신과 민족 정체성을 구축하는 데 큰 영향을 끼쳤습니다. 이순신 장군은 1545년 경상도 거제도에서 태어나 조선 선조 때 거북선을 만들고 뛰어난 작전으로 왜군으로부터 나라를 구해냈습니다. 그의 업적과 일대기는 많은 사람들에게 영감을 주고 있으며, 이를 통해 그의 위업과 헌신을 되짚어보고자 합니다. 이 글에서는 이순신 장군의 어린 시절부터 청년기, 임진왜란과 그의 전술, 그리고 그의 죽음과 그 후에 대해 다루겠습니다. 이순신 장군의 업적과 그의 역사적 위치에 대한 개인적인 평가와 생각도 함께 소개하겠습니다.
## 이순신의 어린 시절

### 출생 및 가문

이순신 장군은 1545년 4월 28일에 서울 건천동 부근에서 태어났습니다. 그의 아버지는 이전에 건천동에서 사는 가문 출신이었고, 어머니는 초계변시 출신이었습니다. 이순신은 덕수 이씨 양반 가문으로 태어나 최고의 교육을 받으며 성장하게 되었습니다. 어린 시절을 한양에서 보낸 이후에는 외가가 있는 충남 아산으로 이사를 가게 되었습니다.

### 어린 시절의 이야기

이순신은 어릴 때부터 영특하고 활달한 아이였습니다. 다른 아이들과 모여 놀 때는 나무를 깎아 화살을 만들어 동리에서 전쟁놀이를 즐겼습니다. 그러나 마음에 거슬리는 사람이 있으면 그의 눈을 쏘려고 했고, 어른들도 그를 꺼려서 군문 앞을 지나려고 하지 않았습니다. 이순신은 자라면서 활을 잘 쏘았으며, 무과에 급제해 관직에 나아가려고 했습니다. 그는 말타고 활쏘기를 잘하고 글씨도 잘 썼습니다. 이런 이순신의 어린 시절은 그의 무인의 기질과 미래의 장군으로서의 자질을 보여주는 시기였습니다.
## 이순신의 청년기

### 군인으로서의 시작

이순신은 청년기에 군인으로서의 시작을 하게 됩니다. 22세의 늦은 나이에 무예를 배우기 시작한 이순신은 뛰어난 실력을 보여주며 무과에 급제하여 본격적인 군인 생활을 시작합니다. 그러나 그의 강직한 성품과 불의에 대한 타협을 하지 않는 태도로 인해 승진이 늦어지고 모함을 받는 등 많은 어려움을 겪게 됩니다. 그러나 그는 항상 반듯하고 청렴한 삶을 살아가며 군인으로서의 역할을 충실히 수행합니다.

### 청년기의 주요 사건들

이순신의 청년기에는 여러 주요 사건들이 있었습니다. 그 중 하나는 1591년 2월 13일에 이순신이 종 6품 현감으로 임명된 사건입니다. 당시 왕이었던 선조는 이순신을 종 3품 전라좌도 수군절도사에 임명하여 군사들을 훈련시키고 거북선을 만들며 강력한 수군을 구축하게 됩니다. 이순신은 이듬해인 1592년에 임진왜란이 일어나자 옥포에서 일본 수군과 첫 해전을 벌여 크게 승리하게 됩니다. 청년기에 이순신은 이미 그의 뛰어난 전략과 전투 능력을 발휘하며 군인으로서의 업적을 쌓아갔습니다.
## 임진왜란과 이순신

### 임진왜란의 배경

임진왜란은 1592년부터 1598년까지 일본의 침략으로 벌어진 전쟁입니다. 이 전쟁은 조선에게 큰 위기를 안겨주었으며, 조선은 일본의 강력한 전력과 무시무시한 기술력에 직면하게 되었습니다. 그러나 이순신 장군은 난처한 상황에서도 독특한 전략을 펼쳐 일본 해군을 무찌르는 데 성공했습니다. 그의 전략과 전투는 조선의 운명을 바꿔놓은 것으로 평가받고 있습니다.

### 이순신의 전략과 전투

이순신 장군은 임진왜란 동안 다양한 전략과 전투를 펼쳤습니다. 그 중에서도 가장 대표적인 전략 중 하나는 조선 해군이 적의 배를 들어올리고 무장해제한 후, 이를 습격하는 방식이었습니다. 이러한 전술은 일본 해군에게 큰 충격을 주며 그들의 전쟁욕구를 꺾는 데에 성공했습니다.

또한 이순신 장군은 조선 함포가 일본 함포보다 사거리가 길다는 점을 파악하고, 적과 거리를 유지하면서 포격하여 손상을 입히는 방법을 사용했습니다. 이를 통해 이순신은 일본 해군을 상대로 원거리 포격전을 펼치며 승리를 거두었습니다.

이순신 장군의 전략과 전투는 그의 뛰어난 전략가로서의 역량을 입증한 것으로 평가받고 있으며, 그의 업적은 임진왜란의 결과를 좌우하는 중요한 역할을 했습니다.
## 이순신의 전술과 발명

### 거북선의 개발

이순신 장군의 독창성은 군사 기술로 확장되어 혁신적인 "거북선"을 도입했습니다. 중무장한 이 함선은 튀어나온 가시와 독특한 거북이 모양의 구조로 설계되어 적의 공격에 거의 뚫을 수 없게 만들었습니다. 거북선의 혁신적인 설계는 한국의 해상 패권과 해전의 효율성에 크게 기여했습니다.

### 이순신의 해전 전술

이순신 장군은 다양한 해전에서 뛰어난 전술을 펼쳤습니다. 그의 전술은 적의 약점과 두려움을 이용하는 심리전의 달인이었습니다. 그는 적의 움직임을 예측하고 변화하는 상황에 적응하며 취약성을 활용하는 그의 전술은 많은 승리를 이끌어냈습니다. 이순신 장군의 전략적 탁월함은 전장을 넘어 확장되었으며, 그의 전술은 한국의 주권을 수호하는 데 중요한 역할을 했습니다.
## 이순신의 죽음과 그 후

### 이순신의 마지막 전투와 죽음

이순신 장군의 마지막 전투는 노량해전으로 알려져 있습니다. 1598년 11월 19일, 이순신은 왜군의 탄환에 맞아 전사하게 됩니다. 그의 죽음은 자살이라는 설이 있지만, 이는 비판과 논란의 여지가 있는 주장입니다. 이순신이 전사한 것인지 자살한 것인지에 대한 의견은 여전히 분분합니다. 그러나 이순신이 전사한 것으로 받아들여지고 있으며, 그의 헌신과 투지는 많은 사람들에게 감동을 주고 있습니다.

### 이순신의 유산과 그의 영향

이순신 장군은 그의 업적과 헌신으로 인해 대한민국의 국가 정신과 민족 정체성에 큰 영향을 미쳤습니다. 그의 지혜롭고 용감한 전략은 수많은 해전에서 승리를 이끌어냄으로써 조선의 위기를 극복했습니다. 이순신의 죽음은 조선의 사기를 떨어뜨리고, 왜군을 끝까지 추격하여 섬멸하지 않은 이유로 인해 노량해전이 마지막 전투가 된 것으로 평가되고 있습니다.

이순신의 유산은 그의 전략과 전술, 그리고 거북선과 같은 혁신적인 발명물로 이어지고 있습니다. 그의 업적은 조선의 해군력을 강화시키고, 일본의 침략을 막는 데 큰 역할을 했습니다. 이순신의 헌신적인 삶과 용맹한 업적은 영원히 기억될 것이며, 그의 영향력은 현재에도 이어지고 있습니다. 이순신 장군은 조선시대의 영웅으로서 그의 기록이 후세에 전해지며 우리들에게 영감과 자존감을 선사합니다.
## 결론

이순신 장군은 조선시대의 영웅으로서 그의 업적과 헌신으로 인해 대한민국의 국가 정신과 민족 정체성에 큰 영향을 미쳤습니다. 그의 지혜롭고 용감한 전략은 수많은 해전에서 승리를 이끌어냄으로써 조선의 위기를 극복했습니다. 이순신의 죽음은 조선의 사기를 떨어뜨리고, 왜군을 끝까지 추격하여 섬멸하지 않은 이유로 인해 노량해전이 마지막 전투가 된 것으로 평가되고 있습니다.

이순신의 유산은 그의 전략과 전술, 그리고 거북선과 같은 혁신적인 발명물로 이어지고 있습니다. 그의 업적은 조선의 해군력을 강화시키고, 일본의 침략을 막는 데 큰 역할을 했습니다. 이순신의 헌신적인 삶과 용맹한 업적은 영원히 기억될 것이며, 그의 영향력은 현재에도 이어지고 있습니다. 이순신 장군은 조선시대의 영웅으로서 그의 기록이 후세에 전해지며 우리들에게 영감과 자존감을 선사합니다. 이순신 장군의 업적과 그의 역사적 위치, 그리고 개인적인 평가와 생각은 우리에게 이순신의 위대함과 그의 영향력을 상기시키는 역할을 합니다. 그의 이야기는 불굴의 인간 정신이 전략적 비전과 결합될 때 가장 어려운 상황도 극복할 수 있다는 것을 보여줍니다. 이순신 장군은 우리에게 영원한 영감을 주는 위대한 인물입니다.
'''
query = '이순신 장군은 어떤 사람인가요?'

In [50]:
result = draft_chunk_tool._run(draft=draft, query=query)



> Entering new LLMChain chain...
Prompt after formatting:

I want you to look at the draft and extract a part that corresponds to the user's query.
The extracted part should not differ from the original draft.
The extracted part is organized in paragraph.
Just say the extracted part. But if you don't have any, don't say anything and don't explain.

Draft:

# 서론

이순신 장군에 대한 간략한 소개

이순신 장군은 조선시대의 영웅이자 조선해군의 명장으로, 그의 삶과 전투 업적은 오늘날 대한민국의 국가 정신과 민족 정체성을 구축하는 데 큰 영향을 끼쳤습니다. 이순신 장군은 1545년 경상도 거제도에서 태어나 조선 선조 때 거북선을 만들고 뛰어난 작전으로 왜군으로부터 나라를 구해냈습니다. 그의 업적과 일대기는 많은 사람들에게 영감을 주고 있으며, 이를 통해 그의 위업과 헌신을 되짚어보고자 합니다. 이 글에서는 이순신 장군의 어린 시절부터 청년기, 임진왜란과 그의 전술, 그리고 그의 죽음과 그 후에 대해 다루겠습니다. 이순신 장군의 업적과 그의 역사적 위치에 대한 개인적인 평가와 생각도 함께 소개하겠습니다.
## 이순신의 어린 시절

### 출생 및 가문

이순신 장군은 1545년 4월 28일에 서울 건천동 부근에서 태어났습니다. 그의 아버지는 이전에 건천동에서 사는 가문 출신이었고, 어머니는 초계변시 출신이었습니다. 이순신은 덕수 이씨 양반 가문으로 태어나 최고의 교육을 받으며 성장하게 되었습니다. 어린 시절을 한양에서 보낸 이후에는 외가가 있는 충남 아산으로 이사를 가게 되었습니다.

### 어린 시절의 이야기

이순신은 어릴 때부터 영특하고 활달한 아이였습니다. 다른 아이들과 모

In [51]:
print(result)

이순신 장군은 조선시대의 영웅이자 조선해군의 명장으로, 그의 삶과 전투 업적은 오늘날 대한민국의 국가 정신과 민족 정체성을 구축하는 데 큰 영향을 끼쳤습니다. 이순신 장군은 1545년 경상도 거제도에서 태어나 조선 선조 때 거북선을 만들고 뛰어난 작전으로 왜군으로부터 나라를 구해냈습니다. 그의 업적과 일대기는 많은 사람들에게 영감을 주고 있으며, 이를 통해 그의 위업과 헌신을 되짚어보고자 합니다. 이 글에서는 이순신 장군의 어린 시절부터 청년기, 임진왜란과 그의 전술, 그리고 그의 죽음과 그 후에 대해 다루겠습니다. 이순신 장군의 업적과 그의 역사적 위치에 대한 개인적인 평가와 생각도 함께 소개하겠습니다.
